# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [7]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [8]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [9]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
neighborhood_locations = pd.read_csv(file_path)


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [18]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    housing_units_per_year = sfo_data.groupby('year')['housing_units'].mean()

    housing_hvplot = housing_units_per_year.hvplot.bar(x='year', 
                                                    y='housing_units', 
                                                    title='Housing Units For Sale Per Year',
                                                    xlabel='year',
                                                    ylabel='Units For Sale',
                                                    ylim=(36000, 42000))
    return housing_hvplot


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    average_gross_rent_per_year = sfo_data.groupby('year')['gross_rent'].mean()
    
    average_rent_hvplot = average_gross_rent_per_year.hvplot.bar(x='year',
                                                                 y='gross_rent',
                                                                 title= 'Average Gross Rent in San Francisco Per Year',
                                                                 xlabel='year',
                                                                 ylabel= 'Average Gross Rent')
    return average_rent_hvplot
    



def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_sales_price_per_year = sfo_data.groupby('year')['sale_price_sqr_foot'].mean()
    
    average_sales_price_per_year_hvplot = average_sales_price_per_year.hvplot.line(x='year',
                                                                                   y='sale_price_sqr_foot',
                                                                                   xlabel='Year',
                                                                                   ylabel='Price Per Square Foot'
                                                                                  )
    return average_sales_price_per_year_hvplot



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    average_price_by_neighborhood = sfo_data.groupby('year','neighborhood')['sale_price_sqr_foot'].mean()
    
    average_price_by_neighborhood_hvplot = average_price_by_neighborhood.hvplot.line(x='year',
                                                                                     y='sale_price_sqr_foot',
                                                                                     xlabel= 'Year',
                                                                                     ylabel='Price Per Square Foot'
                                                                                    )
    return average_price_by_neighborhood_hvplot



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_10_most_expensive = sfo_data.groupby('neighborhood')[['sale_price_sqr_foot','housing_untis','gross_rent']].mean()
    
    top_10_most_expensive_sorted = top_10_most_expensive.sort_values(by='sale_price_sqr_foot', ascending=False)
    
    top_10_most_expensive_sorted_hvplot = top_10_most_expensive_sorted['sale_price_sqr_foot'][:10].hvplot.bar(x='neighbprhood',
                                                                                                              y='sale_price_sqr_foot',
                                                                                                              rot=45,
                                                                                                              title='Top 10 Most Expensive Neighborhoods',
                                                                                                              xlabel='neighborhood',
                                                                                                              ylabel='Price Per Square Foot',
                                                                                                              ylim=((500,1000)))
    return top_10_most_expensive_sorted_hvplot
        


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    rent_vs_sale_price = sfo_data.groupby('neighborhood')[['sale_price_sqr_foot','gross_rent']].mean()
    
    rent_vs_sale_price_sorted = rent_vs_sale_price.sort_values(by=['sale_price_sqr_foot','gross_rent'], ascending=False)
    
    rent_vs_sale_price_sorted_hvplot = rent_vs_sale_price_sorted ['sale_price_sqr_foot','gross_rent'].hvplot.bar(x='neighbprhood',
                                                                                                              y='sale_price_sqr_foot',
                                                                                                              rot=45,
                                                                                                              title='Rent VS Sale Price',
                                                                                                              xlabel='neighborhood',
                                                                                                              ylabel='Price Per Square Foot',
                                                                                                                 ylim=((500,1000)))
                                                                      
    return rent_vs_sale_price_sorted_hvplot
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    avgerage_values = sfo_data.groupby('neighborhood')[['sale_price_sqr_foot', 'housing_units', 'gross_rent']].mean()
    avgerage_values_sorted = avgerage_values.sort_values(by='sale_price_sqr_foot', ascending=False)
    
    parallel_coordinates = px.parallel_coordinates(avgerage_values_sorted[:10], 
                                          color='sale_price_sqr_foot')
    
    return parallel_coordinates



def parallel_categories():
    """Parallel Categories Plot."""
    
    avgerage_values_2 = sfo_data.groupby('neighborhood')[['sale_price_sqr_foot', 'housing_units', 'gross_rent']].mean()
    avgerage_values_sorted_2= avgerage_values_2.sort_values(by='sale_price_sqr_foot', ascending=False)
    
    parallel_categories = px.parallel_categories(avgerage_values_sorted_2[:10], 
                                    dimensions=[avgerage_values_sorted_2[:10].index, 
                                                'sale_price_sqr_foot', 
                                                'housing_units', 
                                                'gross_rent'
                                               ],
                                    color='sale_price_sqr_foot',
                                    labels={'sale_price_sqr_foot': 'Sales Price', 
                                            'housing_units': 'Available Units',
                                            'gross_rent': 'Gross Rent'
                                            }
                                   )
    
    return parallel_categories




def neighborhood_map():
    """Neighborhood Map."""

    avgerage_values = sfo_data.groupby('neighborhood')[['sale_price_sqr_foot', 'housing_units', 'gross_rent']].mean()
    location = df_neighborhood_locations.rename(columns={'Neighborhood': 'neighborhood'})
    location_indexed = locations.set_index('neighborhood', inplace=False)
    location_avgerage_values = pd.concat([avgerage_values, location_indexed], axis='columns', join='inner')
    
    neighborhood_map = px.scatter_mapbox(
    location_avgerage_values,
    lat='Lat',
    lon='Lon',
    size='housing_units',
    color='gross_rent',
    text=location_avgerage_values.index,
    zoom=10.5,
    labels={'Lat': 'Latitude', 
            'Lon': 'Longitude', 
            'gross_rent': 'Gross Rent', 
            'housing_units': 'Housing Units',
            'neighborhood': 'Neighborhood'},
    title='San Francisco CA Neighborhoods'
        )
    
    return neighborhood_map


def sunburst():
    #"""Sunburst Plot."""
    sunburst = sfo_data.groupby('neighborhood')[['sale_price_sqr_foot', 'housing_units', 'gross_rent']].mean()
  
    sunburst_plot= px.sunburst(sunburst, path=['sale_price_sqr_foot','housing_units', 'gross_rent'], 
                  values='gross_rent')

    return sunburst_plot


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [19]:
# Create a Title for the Dashboard
variable_1 = housing_units_per_year()
variable_2 = average_gross_rent()
parallel_categories = parallel_categories()
parallel_coordinates = parallel_coordinates()


housing_row = pn.Row(housing_units_per_year())

prices_column = pn.Column(average_gross_rent(), average_sales_price())
neighborhood_column = pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods())
parallel_column = pn.Column(parallel_categories(), parallel_coordinates())


# Create a tab layout for the dashboard
my_panel = pn.Tabs(
        ("Housing Units", housing_row),
        ("Rent & Sales Prices", price_column),
        ("Neighborhoods", neighborhood_column),
        ("Parallel Charts", parallel_column),
        ("Map", neighborhood_map))


# Create the dashboard
print(type(variable_1), type(variable_2), type(parallel_categories), type(parallel_coordinates)) 

ValueError: No axis named neighborhood for object type <class 'pandas.core.frame.DataFrame'>

## Serve the Panel Dashboard

In [6]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()